In [ ]:
%pip install psycopg2

In [ ]:
import os

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:

import psycopg2

In [ ]:
#find csv files in my current working directory and isolate them
import contextlib

csv_files = [file for file in os.listdir(os.getcwd()) if file.endswith("csv")]
csv_files

In [ ]:
#make a new directory and move them There
dataset_dir = "AdventureWorksDW"
with contextlib.suppress(Exception):
    mkdir = f"mkdir {dataset_dir}"
    os.system(mkdir)



In [ ]:
import shutil


In [ ]:
with contextlib.suppress(Exception):
    for csv in csv_files:
        shutil.move(csv,dataset_dir)
       
        


Create pandas dataframe from csv file

In [ ]:
data_path = f'{os.getcwd()}/{dataset_dir}/'
try:
    df = {file: pd.read_csv(data_path+file) for file in csv_files}
except UnicodeDecodeError:
    df = {file: pd.read_csv(data_path+file,encoding="ISO-8859-1") for file in csv_files}


Clean tables names and column names

In [ ]:
for k in csv_files:
    dataframe=df[k]
    clean_tbl_name=k.lower().replace(" ","_").replace("?","") \
        .replace("-","_").replace(r"/","_").replace("\\","_").replace("%","_") \
            .replace(")","").replace(",","").replace("$","").replace(":","")
    #remove.csv extension
    tbl_name = f'{clean_tbl_name.split(".")[0]}'
    
    dataframe.columns = [x.lower().replace(" ", "_").replace("?", "")
                         .replace("-", "_").replace(r"/", "_").replace("\\", "_").replace("%", "_") \
                             .replace("(","")
                         .replace(")", "").replace(",", "").replace("$", "") for x in dataframe.columns]

    #replacement dictionary that maps pandas dtypes to sql dtypes
    replacements={
        "object": "varchar",
        "float64": "float",
        "int64": "int",
        "datetime64": "timestamp",
        "timedelta64": "timestamp",
        "string": "varchar",

    }

    #table schema
    col_str = ", ".join(f"{n} {d}" for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(replacements)))
    #adding the db connection
    host = "127.0.0.1"
    port = "5432"
    dbname = "Identification_clients_gps"
    user = "postgres"
    password = "Candycove456"
    conn_string = f"host={host} dbname={dbname} user={user} password={password}"
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    #drop table with same name
    cursor.execute(f"drop table if exists {tbl_name};")
  
    #create table
    cursor.execute("create table %s (%s);" % (tbl_name, col_str))
    #insert values to table
    dataframe.to_csv(k,header=dataframe.columns,index=False,encoding="utf-8")
    my_file=open(k)
    SQL_STATEMENT="""
    COPY %s FROM STDIN WITH 
    CSV 
    HEADER 
     DELIMITER AS ','
    """
    cursor.copy_expert(sql=SQL_STATEMENT % tbl_name,file=my_file)
    cursor.execute("grant select on table %s to public" % tbl_name)
    conn.commit()
    conn.close()
    
